In [10]:
import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog
import numpy as np
import tensorflow
from tensorflow.keras.applications import ResNet50, vgg16
from tensorflow.keras.applications.vgg16 import decode_predictions
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import random
import re
import nltk
from nltk.corpus import stopwords, wordnet

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Input, Conv2D, Dropout, LSTM, TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence, image

from gensim.models import Word2Vec, KeyedVectors


In [11]:
from tensorflow.keras.preprocessing.image import img_to_array

In [13]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
import os

# --- Global Variables ---
img_label = None
loaded_img = None
img_path = None

# --- Load image from file system ---
def load_img():
    global img_label, loaded_img, img_path
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png *.jpeg")])
    if file_path:
        img_path = file_path
        img = Image.open(file_path)
        img = img.resize((400, 400))
        loaded_img = ImageTk.PhotoImage(img)
        
        if img_label:
            img_label.configure(image=loaded_img)
            img_label.image = loaded_img
        else:
            img_label = tk.Label(frame, image=loaded_img)
            img_label.image = loaded_img
            img_label.pack()

# --- Dummy Caption Image Function ---
def classify():
    if not img_path:
        result_label.config(text="Please load an image first.")
        return
    # Here you can process the image and predict using model1
    result_label.config(text="Caption (Placeholder): A dog playing in the park.")

# --- Dummy Recommend Captions Function ---
def recommend():
    if not img_path:
        result_label.config(text="Please load an image first.")
        return
    # Here you can process image features and predict tags using model2
    result_label.config(text="Recommendations (Placeholder): #dog #nature #sunnyday")

# --- Tkinter GUI Setup ---
root = tk.Tk()
root.title('Image Captioning')
# root.iconbitmap('class.ico')  # Optional
root.resizable(False, False)

# Title Label
tk.Label(root, text="Portable Image Classifier", padx=25, pady=6, font=("", 12)).pack()

# Canvas and Frame for Image
canvas = tk.Canvas(root, height=700, width=1100, bg='grey')
canvas.pack()

frame = tk.Frame(root, bg='white')
frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)

# Buttons
tk.Button(root, text='Choose Image', padx=35, pady=10, fg="black", bg="grey", command=load_img).pack(side=tk.LEFT)
tk.Button(root, text='Caption Image', padx=35, pady=10, fg="black", bg="grey", command=classify).pack(side=tk.RIGHT)
tk.Button(root, text='Recommend Captions', padx=35, pady=10, fg="black", bg="grey", command=recommend).pack(side=tk.RIGHT)

# Caption Output
result_label = tk.Label(root, text="", font=("Arial", 12), wraplength=600, bg='white')
result_label.pack(pady=10)

# Load Models
try:
    model1 = tf.keras.models.load_model('saved_model.hp5')  # Captioning model
    model2 = tf.keras.models.load_model('model_text_categorize.h5')  # Tag recommender
except Exception as e:
    print("Model loading error:", e)

# Start GUI loop
root.mainloop()


Model loading error: Unrecognized keyword arguments: ['batch_shape']


In [6]:
def load_img():
    global img, image_data
    for img_display in frame.winfo_children():
        img_display.destroy()

    image_data = filedialog.askopenfilename(initialdir="/", title="Choose an image",
                                       filetypes=(("all files", "*.*"), ("png files", "*.png")))
    basewidth = 250 # Processing image for dysplaying
    img = Image.open(image_data)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    file_name = image_data.split('/')
    panel = tk.Label(frame, text= str(file_name[len(file_name)-1]).upper()).pack()
    panel_image = tk.Label(frame, image=img).pack()

In [7]:
def classify():
    global category
    #original = Image.open(image_data)
    original = image.load_img(image_data, target_size=(224,224,3))
    #original = original.resize((224, 224,3
    original = img_to_array(original)
    original = np.expand_dims(original, axis=0)
    resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3),pooling='avg')
    test_img = resnet.predict(original).reshape(2048)
    with open('word_2_indices.pickle', 'rb') as handle:
        word_2_indices_data = pickle.load(handle)
    word_2_indices = word_2_indices_data
    with open('indices_2_word.pickle', 'rb') as handle:
        indices_2_word_data = pickle.load(handle)
    indices_2_word = indices_2_word_data
    max_len = 40
    start_word = ["<start>"]
    while True:
        par_caps = [word_2_indices[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
        preds = model1.predict([np.array([test_img]), np.array(par_caps)])
        word_pred = indices_2_word[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
    label= ' '.join(start_word[1:-1])
    model_w2v = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz',binary=True,limit=1000000)
    description = label
    description = description.replace(r'\d+','')
    spec_chars = ["!",'"',"#","%","&","'","(",")",
                "*","+",",","-",".","/",":",";","<",
                "=",">","?","@","[","\\","]","^","_",
                "`","{","|","}","~","–"]
    for char in spec_chars:
        description = description.replace(char, ' ')
    word_list_t = description.lower().split() 
    filtered_words = [word for word in word_list_t if word not in stopwords.words('english')]
    text = ' '.join(filtered_words)
    description = text
    description_tokens = list(description.split(" "))
    token_list=[]
    for i in description_tokens:
        if len(i) > 3 and i in model_w2v.vocab:
            token_list.append(i)
        else:
            continue
    description_tokens_filtered = token_list
    _arrays = np.zeros((1, 300))
    vec = np.zeros(300).reshape((1, 300))
    count = 0
    for word in description_tokens_filtered:
        vec += model_w2v[word].reshape((1, 300))
        count += 1.
    if count != 0:
        vec /= count
    _arrays[0,:] = vec
    vectorized_array = pd.DataFrame(_arrays)
    pred = model2.predict([vectorized_array.iloc[:,0:300]])
    value = np.argmax(pred,axis=-1)
    labels = ['adventure','art and music','food','history','manufacturing','nature','science and technology','sports','travel']
    category = labels[value.item()]
    result = tk.Label(frame, text= str('Caption:') + str(label) + '\n' + str('Predicted Label :') + str(category)).pack()


In [8]:
def recommend():
    captions = pd.read_csv('captions.csv')
    is_category = captions['Category'].str.lower()== category
    df = captions.loc[is_category]
    df.columns = ('Category', 'Caption')
    df.head()
    caption_list = df['Caption']
    result = tk.Label(frame, text = str('Recommended Captions for the label:') + str('\n')).pack()
    count = 1
    for caption in caption_list:
        result = tk.Label(frame, text = str(count) + str('.') + str(caption)).pack()
        count = count+1